<a href="https://colab.research.google.com/github/antoniovfonseca/agentic-ai-global-lulc/blob/main/notebooks/overview-glance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/antoniovfonseca/agentic-ai-global-lulc/blob/main/notebooks/overview-glance.ipynb)

## **1. Import of geospatial processing and visualization modules.**
---



In [ ]:
# Enter your Google Cloud Project ID
project_id = "ee-antoniovicgf"

In [ ]:
import os
import sys
import warnings

# Filter future warnings to keep output clean
warnings.simplefilter(action='ignore', category=FutureWarning)

# Install geemap dependency
print("Installing dependencies...")
!pip install geemap -q

# Define repository details
repo_url = 'https://github.com/antoniovfonseca/agentic-ai-global-lulc.git'
repo_name = 'agentic-ai-global-lulc'

# Clone or update the repository
if not os.path.exists(repo_name):
    print(f"Cloning {repo_name}...")
    !git clone {repo_url}
else:
    print(f"Updating {repo_name}...")
    !cd {repo_name} && git pull

# Add the notebooks directory to system path to allow imports
notebooks_path = os.path.join(repo_name, 'notebooks')
if notebooks_path not in sys.path:
    sys.path.append(os.path.abspath(notebooks_path))
    print(f"Added {notebooks_path} to system path.")

If you are running this notebook in Google Colab, please uncomment and run the following cell to install the required packages.

## **2. Earth Engine session authentication and initialization.**


---
See the Google instructions [here](https://developers.google.com/earth-engine/guides/auth).


In [ ]:
import ee

# Authenticate and initialize earth engine
try:
    # Try initializing with the provided project id or default
    if project_id:
        ee.Initialize(project=project_id)
    else:
        ee.Initialize()
    print("Earth Engine initialized successfully.")

except Exception:
    # Trigger authentication flow if initialization fails
    print("Authentication required...")
    ee.Authenticate()

    try:
        # Retry initialization after authentication
        if project_id:
            ee.Initialize(project=project_id)
        else:
            ee.Initialize()
        print("Earth Engine initialized successfully.")

    except ee.EEException:
        # Fallback to manual input if no project is found
        print("No default project found.")
        manual_id = input("Please enter your Google Cloud Project ID: ")
        ee.Initialize(project=manual_id)
        print("Earth Engine initialized successfully.")

In [ ]:
try:
    # Import the visualization function from utils
    from utils import get_glance_map
    print("Success: get_glance_map function imported.")
except ImportError as e:
    print(f"Error: Could not import utils. Details: {e}")

## **3. Definition of the remote GLANCE dataset source.**

In [ ]:
# 2. Import the custom function from the utils file
from utils import get_glance_map

# 3. Define parameters
target_year = 2015

# 4. Run the function imported from utils.py
glance_map = get_glance_map(target_year)

# 5. Display the map
if glance_map:
    display(glance_map)

## **4. Change Component Analysis (CCA)**


---



#### 4.1 Upload and Apply mask

In [ ]:
from google.colab import files

# Section 2: Upload Raster File
print("Please upload your raster file (.tif):")

# This triggers the file upload widget
uploaded = files.upload()

# Automatically get the filename of the uploaded file
if uploaded:
    raster_filename = list(uploaded.keys())[0]
    print(f"\nFile '{raster_filename}' uploaded successfully.")
else:
    raise ValueError("No file uploaded. Please run the cell again and select a file.")

In [13]:
import os
import logging
import ee
import geemap
import numpy as np
import rasterio
from rasterio.warp import reproject
from rasterio.warp import Resampling
from rasterio.warp import transform_bounds

# Suppress warnings
logging.getLogger('googleapiclient.http').setLevel(logging.ERROR)
logging.getLogger('urllib3.connectionpool').setLevel(logging.ERROR)

# Install geedim if missing
try:
    import geedim
except ImportError:
    print("Installing geedim...")
    !pip install geedim -q

# Define directories
base_dir = 'glance_data'
raw_dir = os.path.join(base_dir, 'raw')
masked_dir = os.path.join(base_dir, 'masked')

# Create directories if they do not exist
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(masked_dir, exist_ok=True)

# Define processing parameters
years = [2005, 2010, 2015]

# Get geometry from local mask
with rasterio.open(raster_filename) as src:
    left, bottom, right, top = src.bounds
    if src.crs != 'EPSG:4326':
        left, bottom, right, top = transform_bounds(
            src.crs,
            'EPSG:4326',
            left,
            bottom,
            right,
            top
        )
    region = ee.Geometry.Rectangle([left, bottom, right, top])

# Process each year
for year in years:
    filename = f"glance_{year}.tif"
    raw_path = os.path.join(raw_dir, filename)
    masked_path = os.path.join(masked_dir, filename)

    # Skip if final masked file exists
    if os.path.exists(masked_path):
        print(f"Skipping {year} (masked file exists).")
        continue

    print(f"Processing year {year}...")

    # 1. Download raw data to raw folder
    if not os.path.exists(raw_path):
        try:
            image = ee.ImageCollection("projects/GLANCE/DATASETS/V001") \
                .filterDate(f"{year}-01-01", f"{year}-12-31") \
                .mosaic() \
                .select("LC") \
                .clip(region)

            geemap.download_ee_image(
                image,
                filename=raw_path,
                region=region,
                scale=30,
                crs='EPSG:4326',
                unmask_value=0,
                num_threads=4
            )
        except Exception as e:
            print(f"Error downloading {year}: {e}")
            continue

    # 2. Apply local mask and save to masked folder
    try:
        with rasterio.open(raster_filename) as mask_src:
            mask_data = mask_src.read(1)
            kwargs = mask_src.profile.copy()

            with rasterio.open(raw_path) as glance_src:
                glance_aligned = np.zeros(
                    mask_data.shape,
                    dtype=np.uint8
                )

                reproject(
                    source=rasterio.band(glance_src, 1),
                    destination=glance_aligned,
                    src_transform=glance_src.transform,
                    src_crs=glance_src.crs,
                    dst_transform=mask_src.transform,
                    dst_crs=mask_src.crs,
                    resampling=Resampling.nearest
                )

        # Apply mask logic (1 keeps value, others become 255)
        final_data = np.full(mask_data.shape, 255, dtype=np.uint8)
        valid_pixels = (mask_data == 1)
        final_data[valid_pixels] = glance_aligned[valid_pixels]

        # Update profile for compression
        kwargs.update(
            dtype=rasterio.uint8,
            nodata=255,
            compress='deflate',
            predictor=2,
            tiled=True
        )

        with rasterio.open(masked_path, 'w', **kwargs) as dst:
            dst.write(final_data, 1)

        print(f"Finished: {masked_path}")

    except Exception as e:
        print(f"Error masking {year}: {e}")

Processing year 2005...


  0%|          |0/164 tiles [00:00<?]

Error downloading 2005: Too Many Requests: Request was rejected because the request rate or concurrency limit was exceeded.
Processing year 2010...


  0%|          |0/164 tiles [00:00<?]

Error masking 2010: The height and width of TIFF dataset blocks must be multiples of 16
Processing year 2015...


  0%|          |0/164 tiles [00:00<?]

Error downloading 2015: Too Many Requests: Request was rejected because the request rate or concurrency limit was exceeded.
